In [36]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
import numpy as np

In [2]:
df = pd.read_csv(r"D:\python\projects\car_prize_prediction\Samochody Otomoto")
df.head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor
0,165000,BMW,X5,2015,212000,2993.0,Diesel,313,Automatyczna,5,Czarny
1,15700,Alfa Romeo,GT,2007,212325,1910.0,Diesel,150,Manualna,2,Czarny
2,40900,BMW,Seria 3GeneracjaF30/F31 (2012-),2013,195000,1995.0,Diesel,115,Manualna,5,Srebrny
3,22000,Volkswagen,Golf,2011,254000,1598.0,Diesel,105,Manualna,5,Czarny
4,32800,Opel,Astra,2013,238495,1364.0,Benzyna,140,Manualna,5,Brązowy


In [3]:
labelencoder = LabelEncoder()

In [4]:
for x in df["przebieg"]:
    if len(x) > 10:
        df = df[df["przebieg"] != x]
        
for x in df["cena"]:
    if len(x) > 10:
        df = df[df["cena"] != x]

In [5]:
df["cena"] = pd.to_numeric(df["cena"])
df["marka label"] = labelencoder.fit_transform(df["marka"])
df["model label"] = labelencoder.fit_transform(df["model"])
df["rocznik"] = pd.to_numeric(df["rocznik"])
df["przebieg"] = pd.to_numeric(df["przebieg"])
df["pojemnosc"] = pd.to_numeric(df["pojemnosc"])
df["rodzaj paliwa label"] = labelencoder.fit_transform(df["rodzaj paliwa"])
df["moc"] = pd.to_numeric(df["moc"])
df["skrzynia biegow label"] = labelencoder.fit_transform(df["skrzynia biegow"])
df["liczba drzwi"] = pd.to_numeric(df["liczba drzwi"])
df["kolor label"] = labelencoder.fit_transform(df["kolor"])

In [6]:
df.head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
0,165000,BMW,X5,2015,212000,2993.0,Diesel,313,Automatyczna,5,Czarny,4,599,2,0,20
1,15700,Alfa Romeo,GT,2007,212325,1910.0,Diesel,150,Manualna,2,Czarny,2,250,2,1,20
2,40900,BMW,Seria 3GeneracjaF30/F31 (2012-),2013,195000,1995.0,Diesel,115,Manualna,5,Srebrny,4,508,2,1,56
3,22000,Volkswagen,Golf,2011,254000,1598.0,Diesel,105,Manualna,5,Czarny,44,255,2,1,20
4,32800,Opel,Astra,2013,238495,1364.0,Benzyna,140,Manualna,5,Brązowy,32,79,0,1,14


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2584 entries, 0 to 2589
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cena                   2584 non-null   int64  
 1   marka                  2584 non-null   object 
 2   model                  2584 non-null   object 
 3   rocznik                2584 non-null   int64  
 4   przebieg               2584 non-null   int64  
 5   pojemnosc              2584 non-null   float64
 6   rodzaj paliwa          2584 non-null   object 
 7   moc                    2584 non-null   int64  
 8   skrzynia biegow        2584 non-null   object 
 9   liczba drzwi           2584 non-null   int64  
 10  kolor                  2584 non-null   object 
 11  marka label            2584 non-null   int32  
 12  model label            2584 non-null   int32  
 13  rodzaj paliwa label    2584 non-null   int32  
 14  skrzynia biegow label  2584 non-null   int32  
 15  kolo

In [44]:
X = df[["marka label", "model label", "rodzaj paliwa label", 
        "kolor label", "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]]
y = df["cena"]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [46]:
regressor  = XGBRegressor()

In [47]:
regressor.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [48]:
Y_pred_train = regressor.predict(X_train)

In [49]:
y_pred = regressor.predict(X_test)

In [50]:
def rmsle(y_pred,y_test) :
    error = np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean() ** 0.5
    Acc = 1 - error
    return Acc

print("Accuracy attained on Training Set = ",rmsle(Y_pred_train, y_train))
print("Accuracy attained on Test Set = ",rmsle(y_pred,y_test))

Accuracy attained on Training Set =  0.9624153954419264
Accuracy attained on Test Set =  0.8676574788086888


In [51]:
lr = LinearRegression()
lr.fit(X,y)
lr.score(X,y)

0.6854595383942699

In [52]:
df[df["model"] == "V50"]

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
401,23900,Volvo,V50,2009,232000,1798.0,Benzyna,125,Manualna,5,Czarny,45,575,0,1,20
453,13900,Volvo,V50,2007,310000,1998.0,Diesel,136,Manualna,5,Czarny,45,575,2,1,20
660,20600,Volvo,V50,2011,204000,1560.0,Diesel,115,Manualna,5,Czarny,45,575,2,1,20
1274,8900,Volvo,V50,2007,472256,1560.0,Diesel,109,Manualna,5,Niebieski,45,575,2,1,47
1348,18300,Volvo,V50,2008,250000,1798.0,Benzyna,125,Manualna,5,Inny kolor,45,575,0,1,44
1574,17900,Volvo,V50,2006,188435,1798.0,Benzyna,125,Manualna,5,Czarny,45,575,0,1,20
1633,12000,Volvo,V50,2005,220333,1998.0,Diesel,136,Manualna,5,Niebieski,45,575,2,1,47
1890,12900,Volvo,V50,2007,292800,1560.0,Diesel,109,Manualna,5,Szary,45,575,2,1,63
2407,13999,Volvo,V50,2005,225000,1998.0,Diesel,136,Manualna,5,Niebieski,45,575,2,1,47


In [60]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", "rocznik", 
 "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
volvo = [45, 575, 2, 20, 2005, 260333, 1998.0, 170, 5, 1]
volvo = np.array(volvo).reshape((1,-1))

In [61]:
regressor.predict(volvo)

array([13996.032], dtype=float32)

In [56]:
df[df["model"] == "SX4"]

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
89,31900,Suzuki,SX4,2012,143000,1586.0,Benzyna,120,Manualna,5,Szary,42,485,0,1,63
567,19900,Suzuki,SX4,2010,131551,1586.0,Benzyna,120,Manualna,5,Czarny,42,485,0,1,20
1480,18900,Suzuki,SX4,2006,189000,1490.0,Benzyna+LPG,99,Manualna,5,Czarny,42,485,1,1,20
1578,13500,Suzuki,SX4,2006,257000,1586.0,Benzyna,107,Manualna,5,Czerwony,42,485,0,1,37
1767,16500,Suzuki,SX4,2008,202000,1586.0,Benzyna+LPG,107,Manualna,5,Srebrny,42,485,1,1,56


In [83]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", 
 "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
suzuki = [42, 485, 0, 37, 2007, 140000, 1586.0, 111, 5, 1]
suzuki = np.array(suzuki).reshape((1,-1))

In [84]:
regressor.predict(suzuki)

array([16701.188], dtype=float32)

In [63]:
df[df["model"] == "Fabia"]

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
380,9900,Škoda,Fabia,2003,46000,1390.0,Benzyna,75,Manualna,5,Srebrny,46,218,0,1,56
574,18400,Škoda,Fabia,2009,96900,1598.0,Benzyna,105,Manualna,5,Szary,46,218,0,1,63
589,4950,Škoda,Fabia,2003,325994,1896.0,Diesel,100,Manualna,5,Zielony,46,218,2,1,75
950,24000,Škoda,Fabia,2015,197000,1197.0,Benzyna,90,Manualna,5,Niebieski,46,218,0,1,47
1806,18300,Škoda,Fabia,2011,94000,1197.0,Benzyna,105,Manualna,5,Niebieski,46,218,0,1,47
1818,18500,Škoda,Fabia,2010,191572,1198.0,Benzyna,70,Manualna,5,Czarny,46,218,0,1,20
1981,10900,Škoda,Fabia,2004,86324,1390.0,Benzyna,100,Manualna,5,Czerwony,46,218,0,1,37
2003,33333,Škoda,Fabia,2015,27000,999.0,Benzyna,60,Manualna,5,Inny kolor,46,218,0,1,44
2005,34700,Škoda,Fabia,2015,65000,999.0,Benzyna,60,Manualna,5,Niebieski,46,218,0,1,47
2129,4000,Škoda,Fabia,2004,185400,1198.0,Benzyna,64,Manualna,5,Czerwony,46,218,0,1,37


In [95]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", 
 "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
skoda = [46, 218, 2, 20, 2013, 185000, 1598.0, 70, 5, 1]
skoda = np.array(skoda).reshape((1,-1))

In [96]:
regressor.predict(skoda)

array([17546.09], dtype=float32)